# Suspicious Detection 

## Mount Drive

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## Set the path where dataset is stored

In [0]:
data_root='/content/drive/My Drive/Suspicious Activity Detection/Dataset'

## Set path of all classes and get number of data in each class


In [0]:
suspDir=data_root+'/Suspicious'
fireDir=data_root+'/Fire'
safeDir = data_root+'/Safe'
print("Suspicious Images=",len(os.listdir(suspDir)),
      "\nFire Images=",len(os.listdir(fireDir)),
      "\nSafe Images=",len(os.listdir(safeDir)))

Suspicious Images= 1723 
Fire Images= 160 
Safe Images= 3342


## Install required packages

In [0]:
pip install tensorflow-gpu==2.0.0-beta0

In [0]:
pip install tensorflow_hub

In [0]:
pip install tensorflow

In [0]:
tf.__version__

'2.2.0-rc2'

In [0]:
try:
  %tensorflow_version 2.2.1
except Exception:
  pass

`%tensorflow_version` only switches the major version: 1.x or 2.x.
You set: `2.2.1`. This will be interpreted as: `2.x`.


TensorFlow is already loaded. Please restart the runtime to change versions.


## Import Required Libraries

In [0]:
from __future__ import absolute_import, division, print_function, unicode_literals
import matplotlib.pylab as plt
import tensorflow as tf
import tensorflow_hub as hub
import numpy as np
import pandas as pd

## Code to resize images 

In [0]:
# import cv2
 
# x=suspDir # path to dir where images are stored
# i=0
# for f in os.listdir(x)[1656:]:
#     i+=1
#     img = cv2.imread(os.path.join(x,f))
#     resized = cv2.resize(img, (224,224))
#     cv2.imwrite(x+'/'+str(f),resized)

## Code to generate training and validation generators

In [0]:
IMAGE_SHAPE = (224, 224)
TRAINING_DATA_DIR = str(data_root)
print(TRAINING_DATA_DIR);
datagen_kwargs_val = dict(rescale=1./255, validation_split=.20)
valid_datagen = tf.keras.preprocessing.image.ImageDataGenerator(**datagen_kwargs_val)
valid_generator = valid_datagen.flow_from_directory(
TRAINING_DATA_DIR,
subset="validation",
shuffle=True,
target_size=IMAGE_SHAPE
)
datagen_kwargs_train = dict(rescale=1./255, validation_split=.20,horizontal_flip=True,rotation_range=45,zoom_range=0.2,shear_range=0.2,
                                                                height_shift_range=0.2,width_shift_range=0.2)
train_datagen = tf.keras.preprocessing.image.ImageDataGenerator(**datagen_kwargs_train)
train_generator = train_datagen.flow_from_directory(
TRAINING_DATA_DIR,
subset="training",
shuffle=True,
target_size=IMAGE_SHAPE
)

/content/drive/My Drive/Suspicious Activity Detection/Dataset
Found 1044 images belonging to 3 classes.
Found 4181 images belonging to 3 classes.


## Print all labels and shape of images

In [0]:
image_batch_train, label_batch_train = next(iter(train_generator))
print("Image batch shape: ", image_batch_train.shape)
print("Label batch shape: ", label_batch_train.shape)
dataset_labels = sorted(train_generator.class_indices.items(), key=lambda pair:pair[1])
dataset_labels = np.array([key.title() for key, value in dataset_labels])
print(dataset_labels)

Image batch shape:  (32, 224, 224, 3)
Label batch shape:  (32, 3)
['Fire' 'Safe' 'Suspicious']


## Build the model

### Model is a sequential model containing following layers


1.   Feature vector of pretrained mobile-net v2 model
2.   Dropout layer to reduce overfitting
3.   Dense or FC layer as output layer with softmax activation



In [0]:
model = tf.keras.Sequential([
hub.KerasLayer("https://tfhub.dev/google/tf2-preview/mobilenet_v2/feature_vector/4",
output_shape=[1280],
trainable=False),
tf.keras.layers.Dropout(0.4),
tf.keras.layers.Dense(train_generator.num_classes, activation='softmax')
])
model.build([None, 224, 224, 3])
model.summary()
model.compile(
optimizer='adam',
loss='categorical_crossentropy',
metrics=['acc']
)

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
keras_layer_4 (KerasLayer)   multiple                  2257984   
_________________________________________________________________
dropout_4 (Dropout)          multiple                  0         
_________________________________________________________________
dense_4 (Dense)              multiple                  3843      
Total params: 2,261,827
Trainable params: 3,843
Non-trainable params: 2,257,984
_________________________________________________________________


### Train the model

In [0]:
steps_per_epoch = np.ceil(train_generator.samples/train_generator.batch_size)
val_steps_per_epoch = np.ceil(valid_generator.samples/valid_generator.batch_size)
hist = model.fit(
train_generator,
epochs=20,
verbose=1,
steps_per_epoch=steps_per_epoch,
validation_data=valid_generator,
validation_steps=val_steps_per_epoch
)

Epoch 1/20
131/131 [==============================] - 53s 406ms/step - loss: 0.4660 - acc: 0.8223 - val_loss: 0.2253 - val_acc: 0.9119
Epoch 2/20
131/131 [==============================] - 53s 401ms/step - loss: 0.2262 - acc: 0.9175 - val_loss: 0.2042 - val_acc: 0.9195
Epoch 3/20
131/131 [==============================] - 53s 403ms/step - loss: 0.1883 - acc: 0.9333 - val_loss: 0.1299 - val_acc: 0.9406
Epoch 4/20
131/131 [==============================] - 52s 401ms/step - loss: 0.1778 - acc: 0.9412 - val_loss: 0.1244 - val_acc: 0.9444
Epoch 5/20
131/131 [==============================] - 53s 401ms/step - loss: 0.1744 - acc: 0.9436 - val_loss: 0.1209 - val_acc: 0.9540
Epoch 6/20
131/131 [==============================] - 53s 401ms/step - loss: 0.1562 - acc: 0.9457 - val_loss: 0.1202 - val_acc: 0.9454
Epoch 7/20
131/131 [==============================] - 53s 402ms/step - loss: 0.1628 - acc: 0.9450 - val_loss: 0.1053 - val_acc: 0.9559
Epoch 8/20
131/131 [==============================] - 5

## Save the model

### Path where model is to be saved

In [0]:
h5_model_path='/content/drive/My Drive/Suspicious Activity Detection/suspicious_detection_model.h5'

### Save the model in HDF5 format

In [0]:
model.save("/content/drive/My Drive/Suspicious Activity Detection/suspicious_detection_model.h5")

### Load the HDF5 model from given path

In [0]:
suspicious_detection_model=tf.keras.models.load_model(h5_model_path,custom_objects={'KerasLayer':hub.KerasLayer})

In [0]:
suspicious_detection_model.summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
keras_layer_4 (KerasLayer)   multiple                  2257984   
_________________________________________________________________
dropout_4 (Dropout)          multiple                  0         
_________________________________________________________________
dense_4 (Dense)              multiple                  3843      
Total params: 2,261,827
Trainable params: 3,843
Non-trainable params: 2,257,984
_________________________________________________________________


## Test the model on test data

In [0]:
TEST_DATA_DIR = '/content/drive/My Drive/Suspicious Activity Detection/TestDataset/'
IMG_SHAPE=(224,224)
test_datagen_kwargs = dict(rescale=1./255)
test_datagen = tf.keras.preprocessing.image.ImageDataGenerator(**test_datagen_kwargs)
test_generator = test_datagen.flow_from_directory(
    TEST_DATA_DIR,
    target_size=IMG_SHAPE
)

Found 43 images belonging to 3 classes.


In [0]:
test_image_batch, test_label_batch = next(iter(test_generator))
true_label_ids = np.argmax(test_label_batch, axis=-1)
print(true_label_ids)
print("Test batch shape:", test_image_batch.shape)

[2 1 2 2 1 2 1 0 2 2 2 2 1 0 0 0 1 1 2 1 0 2 0 0 0 2 0 0 0 2 0 1]
Test batch shape: (32, 224, 224, 3)


In [0]:
tf_model_predictions = suspicious_detection_model.predict(test_image_batch)
tf_pred_dataframe = pd.DataFrame(tf_model_predictions)
tf_pred_dataframe.columns = dataset_labels
print("Prediction results for the first elements")
#print(tf_model_predictions)
predicted_ids = np.argmax(tf_model_predictions, axis=-1)
predicted_labels = dataset_labels[predicted_ids]
print(predicted_ids)
print(true_label_ids)

Prediction results for the first elements
[2 1 2 2 1 2 1 0 2 2 2 1 1 0 1 0 1 2 2 1 0 2 0 0 0 2 0 0 0 1 0 1]
[2 1 2 2 1 2 1 0 2 2 2 2 1 0 0 0 1 1 2 1 0 2 0 0 0 2 0 0 0 2 0 1]


In [0]:
for i in range(1):
 test_image_batch, test_label_batch = next(iter(test_generator))
 #print(len(test_image_batch))
 true_label_ids = np.argmax(test_label_batch, axis=-1)
 tf_model_predictions = suspicious_detection_model.predict(test_image_batch)
 tf_pred_dataframe = pd.DataFrame(tf_model_predictions)
 tf_pred_dataframe.columns = dataset_labels
 predicted_ids = np.argmax(tf_model_predictions, axis=-1)
 predicted_labels = dataset_labels[predicted_ids]
 plt.figure(figsize=(10,9))
 plt.subplots_adjust(hspace=0.5)
 for n in range(29):
   plt.subplot(6,5,n+1)
   plt.imshow(test_image_batch[n])
   color = "green" if predicted_ids[n] == true_label_ids[n] else "red"
   plt.title(predicted_labels[n].title(), color=color)
   plt.axis('off')
   _ = plt.suptitle("Model predictions (green: correct, red: incorrect)")